## Homework One:  Regular Expressions, Finite-State Automata, and Lexical Analysis

Please provide answers to these problems by filling in this notebook and submitting it via websubmit by Tuesday 1/30 by midnight.  For the second problem, you must also submit an electronic version of your drawings, either using drawing software or by scanning or photographing hand drawings. Make sure in the latter case that the result is legible; if we can not understand your drawing, it will receive no credit. 

In addition to the above, submit your solution to Lab01 at the same time.

You may submit by Wednesday midnight for a 20% penalty; after that no credit will be given. 

I will drop the lowest homework/lab combination at the end of the term.

<b> Make sure to click Cell->Run All before submitting notebooks!!</b>

## Problem One

Describe precisely and concisely the languages denoted by the following regular 
expressions (note: you will be graded on HOW well you understand the languages 
denoted, e.g., you will receive <i>no credit</i> if you describe the third language as 
"the language of any number of <b>a</b>'s or <b>b</b>'s, followed by an <b>a</b>, 
followed by either an <b>a</b> or a <b>b</b>, followed by either an <b>a</b> or 
a <b>b</b>, followed by either an <b>a</b> or a <b>b</b>"). 
<ol style="list-style-type: lower-alpha;">
      <li>$(1)^\ast 1 (1)^\ast 1 (1)^\ast 1 (1)^\ast 1 (1)^\ast 1 (1)^\ast$</li>
      <li>$0 (0\,|\,1)^\ast 1 (0\,|\,1)^\ast 1 (0\,|\,1)^\ast 0$ </li>
      <li>$(a\,|\,b)^\ast a(a\,|\,b)(a\,|\,b)(a\,|\,b)$   </li>
      <li> $(b\,|\,ab)^\ast(a\,|\,b^\ast)$</li>
      <li>$(aa\,|\,bb)^\ast((ab\,|\,ba)(aa\,|\,bb)^\ast(ab\,|\,ba)(aa\,|\,bb)^\ast)^\ast $</li>
    </ol>

## Solution
<ol style="list-style-type: lower-alpha;">
      <li>The language of all strings over {1} with at least 5 1's</li>
      <li>The language of all strings over {0,1} which starts and ends with 0 with at least 2 1's</li>
      <li>The language of all strings over {a,b} which ends with a length 4 string that starts with a and has only 9 combinations for the last 3 characters. </li>
      <li>The language of all strings over {a,b} which does not have any consecitive a's in a row</li>
      <li>The language of all strings over {a,b} with even number of a's and b's</li>
    </ol>

## Problem Two

For each of the following, draw a DFA that accepts the language stated. Make sure to distinguish the start state (with a ">") and any final states (as double circles). 

<ol style="list-style-type: lower-alpha;">
      <li>The language specified by the RE in Problem 1 (d).   </li>
      <li>All strings of bits that contain <b>010</b> as a substring.</li>
      <li>All binary strings which do NOT contain <b>010</b> as a substring</li>
      <li>All strings over {a,b} with an even number of <b>a</b>'s and an odd number of <b>b</b>'s.</li>
      <li>All strings of symbols from the set { /, $\ast$, ", a } starting with the string <b>/$\ast$</b> and ending in <b>$\ast$/</b>, without an intervening <b>$\ast$/</b> unless it appears inside a pair of quotes, and where all quotes must be matched (i.e., there have to be an even number of quotes). (Note: $\ast$ is the ASCII asterisk, not the Kleene Star operator of regular expressions.) Thus:
      
<pre>   
The following are legal:
    /\*aa\*a\*/                 
    /\*/\*\*/
    /\*\*a\*"\*/"/\*/
    /\*""/\*/             
    /\*a"a\*/a"\*/
    
and the following are illegal:

    /\*a\*/\*/
    /\*a/\*
    /\*"\*/
</pre>

      </li>

    </ol>

Hint: There is a strong relationship between the DFA for (b) and the DFA for (c), can you see it?

# Problem Three:  Lexical Analysis

The job of the lexical analyzer is to take a string representing a program or expression to be evaluated, and turn it into a sequence or list of <i>tokens</i>, which are the smallest pieces of syntax, the "words" of the language. White space (blanks, tabs, new lines, etc.) is ignored. 

We want to distil the words down to their minimal representation: the type or category of token, and any other attributes (such as the value of an integer literal). The categories of tokens, represented as integers, are the following:
<pre>
id = 0
integer = 1
floating = 2
plus = 3
minus = 4
mult = 5
div = 6
lparen = 7
rparen = 8
comma = 9
</pre>
We will represent tokens by tuples, where the first element gives the category, and, if necessary, the second element gives an essential attribute for that token. Some tokens have no attributes, and will be represented by a tuple with one element only.  

We will use the following minimal set, which describes
a simple subset of Python tokens, plus a special error token:
<pre>
(id,s)    an identifier (variable/function) spelled as the string s
(integer,n) an integer numeral with value n (n is an actual int)
(floating,k) a float numeral with value k (k is an actual float)
(plus,)   arithmetic operator +
(minus,)  arithmetic operator -
(mult,)   arithmetic operator *
(div,)    arithmetic operator /
(lparen,) left parenthesis (
(rparen,) right parenthesis )
(comma,)  comma ,
(error,)  error
</pre>


(Note that when there are no additional attributes, we use a tuple consisting of a single element.)

For example, the arithmetic expression "(x + (-4 * 3.4))" would be converted into the following list of tokens:

<pre>
[ (lparen,), (id,'x'), (plus,), (lparen,), (minus,), (integer,4), 
  (mult,), (floating,3.4), (rparen,), (rparen,) ]
</pre>
  
Note carefully that the second element of (floating,3.4) is an actual floating-point value, 3.4, and not the original substring "3.4" of the expression. During lexical analysis, the string representation of numeric literals needs to be converted into an actual number. 

## Creating a Lexical Analyzer using a DFA

### (A) Defining the language of tokens using regular expressions

The first step (which we will do for you) is to specify what the syntax of each category of token is, which we will do with regular expressions:

<blockquote>
<b>id:</b> a non-empty string of digits, lower or upper-case letters, and underscores, but NOT starting with a digit:
<pre>
        [a..zA..Z\_][0..9a..zA..Z_]&#42;
</pre>

<b>int:</b> either a single 0, or a non-empty string of digits NOT starting with a 0:
<pre>
        0 | [1..9][0..9]&#42;
</pre>

<b>float:</b> there are two possibilities: (i) same as an int, but with a decimal point and then 0 or more digits after the decimal point, or (ii) a decimal point followed by 1 or more digits:
<pre>
        ( 0 | [1..9][0..9]&#42; )'.'[0..9]&#42; | '.'[0..9]+     
</pre>

Note that we have put single quotes around the decimal point to emphasize that this is part of the alphabet, and not the same as the periods .. which just show the range of characters in a character class. <br><br>

<b>plus:</b> a single '+':
<pre>
              +
</pre>

The other tokens are also just single characters, similar to plus. 
</blockquote>

### (B) Creating recognizers for each of the categories of token 

To use DFAs to create a lexical analyzer for a programming language, we have to complete several steps:

- Define DFAs equivalent to each of the regular expressions for the token categories, each with a single start state, one or more final states, and a single fail state; in fact, the DFAs for the single-character tokens are completely trivial. 
- Write a recognizer function for each category of token, based on the DFAs; in general, it is easier to use the technique of nested-if statements rather than transition tables, because of the large alphabet (e.g., identifiers involve 63 different characters, each requiring a column in the table). 
- if you wish, you may simplify the recognizers for the single-character tokens by simply testing if the string is equal to that character, e.g., 
<pre>
    def legalPlusToken(s):
        return s == '+'
</pre>


### (C) Creating a lexical analyzer based on the recognizers

To use the token recognizers from the previous step in a real lexical analyzer, we have to solve one problem: You are not taking a string and determining if the whole string is a token, but scanning an entire program or expression from left to right and extracting the tokens you encounter as substrings along the way. 

In the interests of making this lab manageable, and to focus on the more interesting details, we will perform a simple but effective "hack" to facilitate this process:

- Put white space (blanks) between each of the potential tokens in the input string by replacing each single-character token such as '+', '(', etc. with a three-character string consisting of a blank, the token, and another blank. Thus '+' would be replaced by ' + ' This can be easily done using the Python replace(...) function on strings. For example, you would convert "(3+x)" into " (  3  +  x  ) ".

- Separate out the potential tokens (which are the substrings consisting of sequential non-white-space characters) by splitting the string on white space, using the Python split() function. For example, you would convert " (  3  +  x  ) " into ["(", "3", "+", "x", ")"], where all white space is removed and you have only <i>potential</i> tokens left. 

- Apply the recognizers to each of the potential tokens, and if a match is found, produce the appropriate token tuple. You will have to convert numeric tokens from strings to numeric form, and may use the Python functions int(...) and float(...) to do the conversions. For example, you would produce from our running example the list:

        [ (lparen,), (integer,3), (plus,), (id,"x"), (rparen,) ]

where at the appropriate point, you converted "3" into 3. For identifiers, no such conversion is necessary!



## Fill in the following code

In [121]:
# Lexical Analyzer Code

# Token Categories

ident = 0
integer = 1
floating = 2
plus = 3
minus = 4
mult = 5
div = 6
lparen = 7
rparen = 8
comma = 9
error = 10

# Code for pretty-printing tokens

tokenCats = ['ident','integer','floating','plus','minus','mult','div',
             'lparen','rparen','comma','error']

def tokenToString(t):
    if t[0] < 3:
        return '(' + tokenCats[t[0]] + ',' + str(t[1]) + ')'
    else:
        return '(' + tokenCats[t[0]] + ',)'
        
def tokenListToString(lst):
    res = '[ '
    for t in lst[:-1]:
        res = res + tokenToString(t) + ', '
    res = res + tokenToString(lst[-1]) + ' ]'
    return res

# Code for lexer

# Replace pass with your solution code!

# put white space between each token and split into words
def splitTokens(s):
    for ch in ['(',')', ',', '+','-','/', '*']:
        s = s.replace(ch, " "+ ch + " ")
    list1 = s.split();
    return list1;
def isIdToken(s):
    if s == '0':
        return True
    state = 1
    for ch in s:
        if state == 1:
            if 'a' <= ch <= 'z' or 'A' <= ch <= 'Z' or ch == '_':
                state = 2
            else:
                return False
        else:   # state = 2
            if 'a' <= ch <= 'z' or 'A' <= ch <= 'Z' or '0' <= ch <= '9' or ch == '_':
                state = 2
            else:
                return False            
    return ( state == 2 )    
            

                        
def isIntToken(s):
    if s == '0':
        return True
    state = 1 
    for ch in s:
        if state == 1:
            if '1' <= ch <= '9':
                state = 2
            else:
                return False;
        else:
            if '0' <= ch <= '9':
                state = 2;
            else:
                return False
    return (state == 2)
            
    
def isFloatToken(s):
    state = 1 
    for ch in s:
        
        if(state == 1):
            if(isIntToken(ch)):
                state = 2;
            elif(ch == '.'):
                state = 4;
            else:
                return False;
        elif(state == 2):
            if(ch == '.'):
                state = 3;
            else:
                return False;
        elif(state == 3):
            if('0' <= ch <= '9'):
                 state = 3;
            else:
                return False
        elif(state == 4):
            if('0' <= ch <= '9'):
                state = 5;
            else:
                return False
        else:
            if('0' <= ch <= '9'):
                state = 5;
            else:
                return False
        
    return(state == 3 or state == 5)    
            
            
            
    
def isPlusToken(s):
    return s == '+';

def isMinusToken(s):
    return s == '-';

def isMultToken(s):
    return s == '*';

def isDivToken(s):
    return s == '/';

def isCommaToken(s):
    return s == ',';

def isLparanToken(s):
    return s == '(';

def isRparanToken(s):
    return s == ')';


def lexer(s):
    string = [];
    list2 = splitTokens(s);
    for i in range(len(list2)):
        if(isLparanToken(list2[i])):
            string.append((7,));
        elif(isRparanToken(list2[i])):
            string.append((8,));
        elif(isCommaToken(list2[i])):
            string.append((9,));
        elif(isDivToken(list2[i])):
            string.append((6,));
        elif(isMultToken(list2[i])):
            string.append((5,));
        elif(isMinusToken(list2[i])):
            string.append((4,));
        elif(isPlusToken(list2[i])):
            string.append((3,));
        elif(isFloatToken(list2[i])):
            string.append((2, float(list2[i])));
        elif(isIntToken(list2[i])):
            string.append((1, int(list2[i])));
        elif(isIdToken(list2[i])):
            string.append((0, list2[i]));
        else:
            string.append((10,));
    return string;
        

In [122]:
# Tests

print(tokenListToString(lexer('x HiThErE y3Y4 _fl_ag4_ 4flag flag.4')))
print()
print(tokenListToString(lexer('4 2304500 0 03434 00 ')))
print()
print(tokenListToString(lexer('3.14  3.  3.0  0.  .0340  03.4  . ')))
print()
print(tokenListToString(lexer('+-*/(),')))
print()
print(tokenListToString(lexer('(4+(flag_4 / -4) * 3.4 - 0 )')))
print("\nNote: in the first three, the last two tokens are errors and") 
print("the previous are correct; for the last two all are correct.")

[ (ident,x), (ident,HiThErE), (ident,y3Y4), (ident,_fl_ag4_), (error,), (error,) ]

[ (integer,4), (integer,2304500), (integer,0), (error,), (error,) ]

[ (floating,3.14), (floating,3.0), (floating,3.0), (floating,0.0), (floating,0.034), (error,), (error,) ]

[ (plus,), (minus,), (mult,), (div,), (lparen,), (rparen,), (comma,) ]

[ (lparen,), (integer,4), (plus,), (lparen,), (ident,flag_4), (div,), (minus,), (integer,4), (rparen,), (mult,), (floating,3.4), (minus,), (integer,0), (rparen,) ]

Note: in the first three, the last two tokens are errors and
the previous are correct; for the last two all are correct.
